In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Kidney_Papillary_Cell_Carcinoma"
cohort = "GSE68606"

# Input paths
in_trait_dir = "../../input/GEO/Kidney_Papillary_Cell_Carcinoma"
in_cohort_dir = "../../input/GEO/Kidney_Papillary_Cell_Carcinoma/GSE68606"

# Output paths
out_data_file = "../../output/preprocess/Kidney_Papillary_Cell_Carcinoma/GSE68606.csv"
out_gene_data_file = "../../output/preprocess/Kidney_Papillary_Cell_Carcinoma/gene_data/GSE68606.csv"
out_clinical_data_file = "../../output/preprocess/Kidney_Papillary_Cell_Carcinoma/clinical_data/GSE68606.csv"
json_path = "../../output/preprocess/Kidney_Papillary_Cell_Carcinoma/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"caArray_dobbi-00100: Interlaboratory comparability study of cancer gene expression analysis using oligonucleotide microarrays"
!Series_summary	"A key step in bringing gene expression data into clinical practice is the conduct of large studies to confirm preliminary models. The performance of such confirmatory studies and the transition to clinical practice requires that microarray data from different laboratories are comparable and reproducible. We designed a study to assess the comparability of data from four laboratories that will conduct a larger microarray profiling confirmation project in lung adenocarcinomas. To test the feasibility of combining data across laboratories, frozen tumor tissues, cell line pellets, and purified RNA samples were analyzed at each of the four laboratories. Samples of each type and several subsamples from each tumor and each cell line were blinded before being distributed. The laboratories followed a common protocol

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
import pandas as pd
import os
import numpy as np
from typing import Optional, Callable, Dict, Any, List
import json

# 1. Gene Expression Data Availability
# Based on the background information, this dataset appears to contain gene expression data
# It mentions "cancer gene expression analysis using oligonucleotide microarrays" and "Affymetrix Human Genome U133A arrays"
is_gene_available = True

# 2. Variable Availability and Data Type Conversion
# 2.1 Data Availability

# For trait: We're looking for Kidney_Papillary_Cell_Carcinoma
# Looking at histology (key 7), there is "histology: Papillary Serous Adenocarcinoma"
# and row 1 has "disease state: Conventional_Clear_Cell_Renal_Cell_Carcinoma" and "disease state: Recurrent Renal Cell Carcinoma"
# From rows 1 and 7, we can infer information about kidney cancer subtypes
trait_row = 7  # Using histology as it contains more detailed information

# For age: Available in key 6
age_row = 6

# For gender: Available in key 5 (labeled as "Sex")
gender_row = 5

# 2.2 Data Type Conversion Functions

def convert_trait(value: str) -> Optional[int]:
    """
    Convert trait values to binary (0: not papillary carcinoma, 1: papillary carcinoma)
    """
    if value is None or value == "--":
        return None
    
    # Extract the value part after the colon and trim whitespace
    if ":" in value:
        value = value.split(":", 1)[1].strip()
    
    # Check for papillary renal cell carcinoma mentions
    if "papillary" in value.lower() and ("renal" in value.lower() or "kidney" in value.lower()):
        return 1
    # Check specifically for "Papillary Serous Adenocarcinoma" which might be kidney-related in this context
    elif "papillary serous adenocarcinoma" in value.lower():
        return 1
    # Other kidney carcinomas that are non-papillary
    elif ("renal cell carcinoma" in value.lower() or "kidney" in value.lower()) and "carcinoma" in value.lower():
        return 0
    # Not a kidney cancer or unspecified
    else:
        return None

def convert_age(value: str) -> Optional[float]:
    """
    Convert age values to continuous numeric values
    """
    if value is None or value == "--":
        return None
    
    # Extract the value part after the colon and trim whitespace
    if ":" in value:
        value = value.split(":", 1)[1].strip()
    
    try:
        return float(value)
    except ValueError:
        return None

def convert_gender(value: str) -> Optional[int]:
    """
    Convert gender values to binary (0: female, 1: male)
    """
    if value is None or value == "--":
        return None
    
    # Extract the value part after the colon and trim whitespace
    if ":" in value:
        value = value.split(":", 1)[1].strip().lower()
    
    if value == "female":
        return 0
    elif value == "male":
        return 1
    else:
        return None

# 3. Save Metadata
# Determine trait data availability
is_trait_available = trait_row is not None

# Initial filtering and metadata saving
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
# If trait_row is not None, extract and save clinical features
if trait_row is not None:
    # Load clinical data (assuming it was loaded in a previous step)
    # We need to check if the variables are defined and clinical_data exists
    try:
        # Use the geo_select_clinical_features function to extract features
        clinical_features = geo_select_clinical_features(
            clinical_df=clinical_data,
            trait=trait,
            trait_row=trait_row,
            convert_trait=convert_trait,
            age_row=age_row,
            convert_age=convert_age,
            gender_row=gender_row,
            convert_gender=convert_gender
        )
        
        # Preview the extracted features
        print("Preview of extracted clinical features:")
        preview = preview_df(clinical_features)
        print(preview)
        
        # Create the directory if it doesn't exist
        os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
        
        # Save the clinical features to the specified output file
        clinical_features.to_csv(out_clinical_data_file)
        print(f"Clinical features saved to {out_clinical_data_file}")
    except NameError:
        print("Clinical data not available from previous steps.")


Preview of extracted clinical features:
{'GSM1676864': [nan, nan, nan], 'GSM1676865': [nan, nan, nan], 'GSM1676866': [nan, nan, 0.0], 'GSM1676867': [nan, nan, nan], 'GSM1676868': [nan, nan, nan], 'GSM1676869': [nan, nan, nan], 'GSM1676870': [nan, nan, nan], 'GSM1676871': [nan, nan, nan], 'GSM1676872': [nan, nan, nan], 'GSM1676873': [nan, nan, nan], 'GSM1676874': [nan, 67.0, 1.0], 'GSM1676875': [nan, 66.0, 1.0], 'GSM1676876': [nan, 72.0, 1.0], 'GSM1676877': [nan, 56.0, 0.0], 'GSM1676878': [nan, 48.0, 0.0], 'GSM1676879': [nan, nan, nan], 'GSM1676880': [nan, nan, nan], 'GSM1676881': [nan, nan, nan], 'GSM1676882': [nan, nan, nan], 'GSM1676883': [nan, nan, nan], 'GSM1676884': [nan, nan, nan], 'GSM1676885': [nan, nan, nan], 'GSM1676886': [nan, nan, nan], 'GSM1676887': [0.0, nan, nan], 'GSM1676888': [nan, nan, nan], 'GSM1676889': [nan, nan, nan], 'GSM1676890': [1.0, nan, nan], 'GSM1676891': [nan, nan, nan], 'GSM1676892': [nan, nan, nan], 'GSM1676893': [nan, nan, nan], 'GSM1676894': [nan, nan,

### Step 3: Gene Data Extraction

In [4]:
# 1. Get the file paths for the SOFT file and matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# Add diagnostic code to check file content and structure
print("Examining matrix file structure...")
with gzip.open(matrix_file, 'rt') as file:
    table_marker_found = False
    lines_read = 0
    for i, line in enumerate(file):
        lines_read += 1
        if '!series_matrix_table_begin' in line:
            table_marker_found = True
            print(f"Found table marker at line {i}")
            # Read a few lines after the marker to check data structure
            next_lines = [next(file, "").strip() for _ in range(5)]
            print("First few lines after marker:")
            for next_line in next_lines:
                print(next_line)
            break
        if i < 10:  # Print first few lines to see file structure
            print(f"Line {i}: {line.strip()}")
        if i > 100:  # Don't read the entire file
            break
    
    if not table_marker_found:
        print("Table marker '!series_matrix_table_begin' not found in first 100 lines")
    print(f"Total lines examined: {lines_read}")

# 2. Try extracting gene expression data from the matrix file again with better diagnostics
try:
    print("\nAttempting to extract gene data from matrix file...")
    gene_data = get_genetic_data(matrix_file)
    if gene_data.empty:
        print("Extracted gene expression data is empty")
        is_gene_available = False
    else:
        print(f"Successfully extracted gene data with {len(gene_data.index)} rows")
        print("First 20 gene IDs:")
        print(gene_data.index[:20])
        is_gene_available = True
except Exception as e:
    print(f"Error extracting gene data: {str(e)}")
    print("This dataset appears to have an empty or malformed gene expression matrix")
    is_gene_available = False

print(f"\nGene expression data available: {is_gene_available}")

# If data extraction failed, try an alternative approach using pandas directly
if not is_gene_available:
    print("\nTrying alternative approach to read gene expression data...")
    try:
        with gzip.open(matrix_file, 'rt') as file:
            # Skip lines until we find the marker
            for line in file:
                if '!series_matrix_table_begin' in line:
                    break
            
            # Try to read the data directly with pandas
            gene_data = pd.read_csv(file, sep='\t', index_col=0)
            
            if not gene_data.empty:
                print(f"Successfully extracted gene data with alternative method: {gene_data.shape}")
                print("First 20 gene IDs:")
                print(gene_data.index[:20])
                is_gene_available = True
            else:
                print("Alternative extraction method also produced empty data")
    except Exception as e:
        print(f"Alternative extraction failed: {str(e)}")


Examining matrix file structure...
Line 0: !Series_title	"caArray_dobbi-00100: Interlaboratory comparability study of cancer gene expression analysis using oligonucleotide microarrays"
Line 1: !Series_geo_accession	"GSE68606"
Line 2: !Series_status	"Public on May 07 2015"
Line 3: !Series_submission_date	"May 06 2015"
Line 4: !Series_last_update_date	"Aug 10 2018"
Line 5: !Series_pubmed_id	"15701842"
Line 6: !Series_summary	"A key step in bringing gene expression data into clinical practice is the conduct of large studies to confirm preliminary models. The performance of such confirmatory studies and the transition to clinical practice requires that microarray data from different laboratories are comparable and reproducible. We designed a study to assess the comparability of data from four laboratories that will conduct a larger microarray profiling confirmation project in lung adenocarcinomas. To test the feasibility of combining data across laboratories, frozen tumor tissues, cell lin

Successfully extracted gene data with 22283 rows
First 20 gene IDs:
Index(['1007_s_at', '1053_at', '117_at', '121_at', '1255_g_at', '1294_at',
       '1316_at', '1320_at', '1405_i_at', '1431_at', '1438_at', '1487_at',
       '1494_f_at', '1598_g_at', '160020_at', '1729_at', '1773_at', '177_at',
       '179_at', '1861_at'],
      dtype='object', name='ID')

Gene expression data available: True


### Step 4: Gene Identifier Review

In [5]:
# Looking at the gene identifiers shown in the previous output
# These identifiers (like '1007_s_at', '1053_at', '117_at') are Affymetrix probe IDs
# from the Human Genome U133A array (as mentioned in Series_summary)
# These are not standard human gene symbols and will need to be mapped to gene symbols

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Extract gene annotation data from the SOFT file
print("Extracting gene annotation data from SOFT file...")
try:
    # Use the library function to extract gene annotation
    gene_annotation = get_gene_annotation(soft_file)
    print(f"Successfully extracted gene annotation data with {len(gene_annotation.index)} rows")
    
    # Preview the annotation DataFrame
    print("\nGene annotation preview (first few rows):")
    print(preview_df(gene_annotation))
    
    # Show column names to help identify which columns we need for mapping
    print("\nColumn names in gene annotation data:")
    print(gene_annotation.columns.tolist())
    
    # Check for relevant mapping columns
    if 'GB_ACC' in gene_annotation.columns:
        print("\nThe dataset contains GenBank accessions (GB_ACC) that could be used for gene mapping.")
        # Count non-null values in GB_ACC column
        non_null_count = gene_annotation['GB_ACC'].count()
        print(f"Number of rows with GenBank accessions: {non_null_count} out of {len(gene_annotation)}")
    
    if 'SPOT_ID' in gene_annotation.columns:
        print("\nThe dataset contains genomic regions (SPOT_ID) that could be used for location-based gene mapping.")
        print("Example SPOT_ID format:", gene_annotation['SPOT_ID'].iloc[0])
    
except Exception as e:
    print(f"Error processing gene annotation data: {e}")
    is_gene_available = False


Extracting gene annotation data from SOFT file...


Successfully extracted gene annotation data with 3075191 rows

Gene annotation preview (first few rows):
{'ID': ['1007_s_at', '1053_at', '117_at', '121_at', '1255_g_at'], 'GB_ACC': ['U48705', 'M87338', 'X51757', 'X69699', 'L36861'], 'SPOT_ID': [nan, nan, nan, nan, nan], 'Species Scientific Name': ['Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens'], 'Annotation Date': ['Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014'], 'Sequence Type': ['Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence'], 'Sequence Source': ['Affymetrix Proprietary Database', 'GenBank', 'Affymetrix Proprietary Database', 'GenBank', 'Affymetrix Proprietary Database'], 'Target Description': ['U48705 /FEATURE=mRNA /DEFINITION=HSU48705 Human receptor tyrosine kinase DDR gene, complete cds', 'M87338 /FEATURE= /DEFINITION=HUMA1SBU Human replication factor C, 40-kDa subunit (A1) mRNA, complete cds', "X51757 /FEATURE=cds /

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Decide which columns in the gene annotation dataframe to use for mapping
# Based on the preview, the 'ID' column in gene_annotation contains the probe IDs (e.g., '1007_s_at')
# The 'Gene Symbol' column contains the gene symbols (e.g., 'DDR1 /// MIR4640')
prob_col = 'ID'
gene_col = 'Gene Symbol'

# 2. Get the gene mapping dataframe
print("Creating gene mapping dataframe...")
mapping_df = get_gene_mapping(gene_annotation, prob_col, gene_col)
print(f"Created gene mapping with {len(mapping_df)} entries")
print("First few rows of mapping dataframe:")
print(preview_df(mapping_df))

# 3. Apply gene mapping to convert probe-level measurements to gene expression data
print("\nConverting probe-level measurements to gene expression data...")
gene_data = apply_gene_mapping(gene_data, mapping_df)
print(f"Converted gene expression data with {len(gene_data.index)} genes")
print("First 10 gene symbols after mapping:")
print(gene_data.index[:10].tolist())

# Save the gene data to a file
print(f"\nSaving gene expression data to {out_gene_data_file}...")
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
gene_data.to_csv(out_gene_data_file)
print("Gene expression data saved successfully.")


Creating gene mapping dataframe...
Created gene mapping with 21225 entries
First few rows of mapping dataframe:
{'ID': ['1007_s_at', '1053_at', '117_at', '121_at', '1255_g_at'], 'Gene': ['DDR1 /// MIR4640', 'RFC2', 'HSPA6', 'PAX8', 'GUCA1A']}

Converting probe-level measurements to gene expression data...


Converted gene expression data with 13830 genes
First 10 gene symbols after mapping:
['A1CF', 'A2M', 'A4GALT', 'A4GNT', 'AAAS', 'AACS', 'AADAC', 'AAGAB', 'AAK1', 'AAMDC']

Saving gene expression data to ../../output/preprocess/Kidney_Papillary_Cell_Carcinoma/gene_data/GSE68606.csv...


Gene expression data saved successfully.


### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize gene symbols in the gene expression data
# This was already done and saved in the previous step, verified in the output
# We'll load it to make sure gene_data is available for the rest of the processing
print("\nLoading gene expression data...")
try:
    gene_data = pd.read_csv(out_gene_data_file, index_col=0)
    print(f"Loaded gene data with {gene_data.shape[0]} genes and {gene_data.shape[1]} samples")
except Exception as e:
    print(f"Error loading gene data: {e}")
    gene_data = pd.DataFrame()

# 2. Extract clinical features
print("\nExtracting clinical features...")
try:
    # Get the file paths for the SOFT file and matrix file again
    soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
    
    # Obtain background information and clinical dataframe from matrix file
    background_info, clinical_data = get_background_and_clinical_data(
        matrix_file, 
        prefixes_a=['!Series_title', '!Series_summary', '!Series_overall_design'],
        prefixes_b=['!Sample_geo_accession', '!Sample_characteristics_ch1']
    )
    
    # For diagnostic purposes, check the unique values in each row
    sample_characteristics_dict = get_unique_values_by_row(clinical_data)
    print("Sample Characteristics Dictionary:")
    print(sample_characteristics_dict)
    
    # Based on sample_characteristics_dict:
    # Row 1 contains disease state info
    # Row 2 contains disease location
    # We'll use both to identify kidney cancer samples
    trait_row = 2  # Use disease location row
    
    # Define conversion function for the trait (kidney cancer)
    def convert_trait(value):
        """
        Convert disease location to binary trait values based on kidney location.
        1 for kidney samples, 0 for other locations
        """
        if value is None:
            return None
        
        # Extract the value part after the colon
        if ':' in value:
            value = value.split(':', 1)[1].strip()
        
        # Check if it's a kidney sample
        if 'kidney' in value.lower():
            return 1  # Kidney samples as cases
        else:
            return 0  # Other locations as controls

    # Extract clinical features
    selected_clinical_df = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=None,  # No age data
        convert_age=None,
        gender_row=None,  # No gender data
        convert_gender=None
    )
    
    # Save the clinical data
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    selected_clinical_df.to_csv(out_clinical_data_file)
    print(f"Clinical data extracted and saved to {out_clinical_data_file}")
    
    # For diagnostic purposes
    print("Clinical data preview:")
    print(preview_df(selected_clinical_df))
    
except Exception as e:
    print(f"Error extracting clinical features: {e}")
    selected_clinical_df = pd.DataFrame()

# 3. Link clinical and genetic data
print("\nLinking clinical and genetic data...")
try:
    # Check if both datasets are available
    if not gene_data.empty and not selected_clinical_df.empty:
        # Link clinical and genetic data
        linked_data = geo_link_clinical_genetic_data(selected_clinical_df, gene_data)
        print(f"Linked data shape: {linked_data.shape}")
        
        # 4. Handle missing values
        print("\nHandling missing values...")
        linked_data = handle_missing_values(linked_data, trait)
        print(f"After handling missing values, data shape: {linked_data.shape}")
        
        # 5. Determine whether the trait and demographic features are biased
        print("\nChecking for bias in features...")
        is_biased, linked_data = judge_and_remove_biased_features(linked_data, trait)
    else:
        print("Clinical data or gene data is not available. Cannot proceed with linking.")
        linked_data = pd.DataFrame()
        is_biased = True
except Exception as e:
    print(f"Error in linking data: {e}")
    linked_data = pd.DataFrame()
    is_biased = True

# 6. Final quality validation
print("\nPerforming final validation...")
note = ""
if 'linked_data' not in locals() or linked_data.empty:
    note = "Dataset failed processing: no usable samples remained after filtering."
    is_biased = True
    linked_data = pd.DataFrame()  # Ensure linked_data exists for validation
elif linked_data.shape[0] < 10:
    note = "Dataset has too few samples (<10) after filtering for valid analysis."
    is_biased = True
elif 'is_biased' in locals() and is_biased:
    note = "Dataset has severely biased trait distribution."

is_gene_available = 'gene_data' in locals() and not gene_data.empty
is_trait_available = 'selected_clinical_df' in locals() and not selected_clinical_df.empty

is_usable = validate_and_save_cohort_info(
    is_final=True,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available,
    is_biased=is_biased if 'is_biased' in locals() else True,
    df=linked_data,
    note=note
)

# 7. Save linked data if usable
if is_usable and 'linked_data' in locals() and not linked_data.empty:
    # Create directory if it doesn't exist
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    
    # Save linked data
    linked_data.to_csv(out_data_file)
    print(f"Linked data saved to {out_data_file}")
else:
    print(f"Dataset not usable for {trait} association studies. Data not saved.")


Loading gene expression data...
Loaded gene data with 13830 genes and 137 samples

Extracting clinical features...


Sample Characteristics Dictionary:
{0: ['cell line: H2347', 'cell line: H1437', 'cell line: HCC78', 'cell line: H2087', 'cell line: H2009', 'cell line: --'], 1: ['disease state: --', 'disease state: Leiomyoma', 'disease state: Lung_Adenocarcinoma', 'disease state: Conventional_Clear_Cell_Renal_Cell_Carcinoma', 'disease state: Squamous Cell Carcinoma', 'disease state: Stomach Adenocarcinoma', 'disease state: Large Cell Lymphoma', 'disease state: Malignant Melanoma', 'disease state: Recurrent Renal Cell Carcinoma', 'disease state: Adrenal Cortical Adenoma', 'disease state: Ovarian Adenocarcinoma', 'disease state: Gastrointestinal_Stromal_Tumor', 'disease state: Metastatic Renal Cell Carcinoma', 'disease state: Non neoplastic liver with cirrosis', 'disease state: Malignant G1 Stromal Tumor', 'disease state: melanoma'], 2: ['tumor grading: --', 'tumor grading: G2/pT1pN0pMX', 'tumor grading: G3/pT2pN0pMX', 'tumor grading: G2/pT2pN0pMX', 'tumor grading: G3/pT4pNXpMX'], 3: ['disease stage: --

After handling missing values, data shape: (137, 13831)

Checking for bias in features...
Quartiles for 'Kidney_Papillary_Cell_Carcinoma':
  25%: 0.0
  50% (Median): 0.0
  75%: 0.0
Min: 0.0
Max: 0.0
The distribution of the feature 'Kidney_Papillary_Cell_Carcinoma' in this dataset is severely biased.


Performing final validation...
Dataset not usable for Kidney_Papillary_Cell_Carcinoma association studies. Data not saved.
